<a href="https://colab.research.google.com/github/ProfDoof/advent_of_code/blob/2022/day11.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Day 

## Load Data

In [ ]:
from pathlib import Path

DAY = 11
DATA_FILE = Path.cwd() / 'drive' / 'MyDrive' / 'AdventOfCode' / 'aoc_data' / f'day{DAY}.txt'

data = DATA_FILE.read_text()

## Solution

In [ ]:
from typing import NamedTuple, List, Callable, Set, Union, Tuple
import operator
import re

test = '''
Monkey 0:
  Starting items: 79, 98
  Operation: new = old * 19
  Test: divisible by 23
    If true: throw to monkey 2
    If false: throw to monkey 3

Monkey 1:
  Starting items: 54, 65, 75, 74
  Operation: new = old + 6
  Test: divisible by 19
    If true: throw to monkey 2
    If false: throw to monkey 0

Monkey 2:
  Starting items: 79, 60, 97
  Operation: new = old * old
  Test: divisible by 13
    If true: throw to monkey 1
    If false: throw to monkey 3

Monkey 3:
  Starting items: 74
  Operation: new = old + 3
  Test: divisible by 17
    If true: throw to monkey 0
    If false: throw to monkey 1
'''

class Monkey(NamedTuple):
    id_: int
    items: List[int]
    operation: Callable[[int], int]
    test_and_pass: Callable[[int], int]


def monkey_op(first, op, second, modulus):
    if op == '+':
        op_func = operator.add
    # Multiplication is kinda like set union
    elif op == '*':
        op_func = operator.mul
    else:
        raise Exception(f'An operator that is not supported was input: "{op}"')
    
    first_val = None if first == 'old' else int(first)
    second_val = None if second == 'old' else int(second)
    return lambda old: op_func(first_val if first_val else old, second_val if second_val else old) % modulus


def monkey_test(div_num: int, true_target: int, false_target: int):
    return lambda worry: true_target if worry % div_num == 0 else false_target


i_data = data

# Parsing step
monkeys_setup = []
monkey_divs = set()
monkey_ops = set()
div_num_re = re.compile('Test: divisible by (\d+)')
target_re = re.compile('If .*?: throw to monkey (\d+)')
for monkey in i_data.strip().split('\n\n'):
    lines = monkey.strip().split('\n')
    id_line = lines[0]
    s_items_line = lines[1]
    op_line = lines[2]
    test_line = lines[3]
    true_test_line = lines[4]
    false_test_line = lines[5]

    ignored, id_ = id_line.strip(' :').split(' ')
    ignored, starting_items = s_items_line.split(':')
    starting_items = [int(s_item.strip()) for s_item in starting_items.split(',')]
    ignored, operation = op_line.split(':')
    # m_op = monkey_op(f, o, s)
    m_div_num = int(div_num_re.search(test_line).group(1))
    m_true_target = int(target_re.search(true_test_line).group(1))
    m_false_target = int(target_re.search(false_test_line).group(1))
    # test_and_pass_func = monkey_test(m_div_num, m_true_target, m_false_target)
    monkey_divs.add(m_div_num)

    monkeys_setup.append((id_, starting_items, operation, m_div_num, m_true_target, m_false_target))

monkeys = []
divs_modulus = 1
for factor in monkey_divs:
    divs_modulus *= factor

DEBUG = False
for id_, starting_items, operation, div_num, true_target, false_target in monkeys_setup:
    ignored1, ignored2, f, o, s = operation.split()
    monkeys.append(
        Monkey(
            id_, 
            starting_items, 
            monkey_op(f, o, s, divs_modulus), 
            monkey_test(div_num, true_target, false_target)
            )
        )

# print(monkeys_setup[0])
# print(monkeys[0])
# Simulation step

ROUNDS = 10000
num_times_inspected = [0 for i in range(len(monkeys))]

for round in range(ROUNDS):
    
    # Go through each monkeys turn
    for monkey_idx, monkey in enumerate(monkeys):
        while len(monkey.items) > 0:
            num_times_inspected[monkey_idx] += 1
            # Check item
            old_worry = monkey.items.pop(0)
            # print(f'  Monkey inspects an item with a worry level of {old_worry}.')

            # Inspect item and increase worry
            new_worry = monkey.operation(old_worry)
            # print(f'    Worry level is increased to {new_worry}.')

            # Decrease worry after it survives
            # new_worry = new_worry // 3
            # print(f'    Monkey gets bored with item. Worry level is divided by 3 to {new_worry}.')

            # Monkey tests worry to determine who to pass to
            target = monkey.test_and_pass(new_worry)
            # print(f'    Item with worry level {new_worry} is thrown to monkey {target}.')
            # Monkey throws to target
            monkeys[target].items.append(new_worry)

k = 2
top_k_values = [(0, 0) for i in range(k)]
for monkey_idx, num_items_inspected in enumerate(num_times_inspected):
    if num_items_inspected > top_k_values[0][0]:
        top_k_values[0] = (num_items_inspected, monkey_idx)
        top_k_values = sorted(top_k_values, key=operator.itemgetter(0))

print(num_times_inspected)
monkey_business = 1
for mb, mi in top_k_values:
    monkey_business *= mb

print(monkey_business)

[83587, 29059, 9882, 123613, 152660, 138194, 142807, 123730]
21800916620
